In [1]:
!pip install mne
!pip install matplotlib

In [3]:
import mne
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz
from scipy.signal import butter, lfilter
from scipy.signal import spectrogram

Band pass filter with cut-off 0.5-50 Hz.

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band',analog=True)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [7]:
def cleanAndRaw(filePath):

    good_channels = ['EEG FP1-REF','EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF','EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']
    good_channels = set(good_channels)

    eeg = mne.io.read_raw_edf(filePath)

    eeg = eeg.resample(100, npad='auto')  

    channels = eeg.info.ch_names

    eeg_19 = eeg.copy()
    for channel in channels:

        if channel not in good_channels:

            eeg_19.drop_channels([channel])

    raw_eeg = eeg_19.get_data()

    if raw_eeg.shape[0] == 19 and raw_eeg.shape[1] >= 36000:

        raw_eeg  = raw_eeg - raw_eeg.mean(axis=1,keepdims=True)

        fs = 100
        lowcut = 0.5
        highcut = 50

        t = np.arange(0,raw_eeg.shape[1],1)

        for i in range(19):
            raw_eeg[i] = butter_bandpass_filter(raw_eeg[i], lowcut, highcut, fs, order=5)

        raw_eeg = np.delete(raw_eeg,np.s_[:600], axis=1)    
        raw_eeg = np.delete(raw_eeg,np.s_[30000:], axis=1)    

        return raw_eeg

    else:

        return None

In [5]:
    
def class_spectrogram(clas,filename):
    if clas == 3:
        base = cv2.imread('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/channel__0.png')

        i = 1
        while i < 19:
            im = cv2.imread('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/channel__'+str(i)+'.png')
            base = cv2.vconcat([base,im])

            i += 1

        cv2.imwrite('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c3/'+filename+'.png',base)

Creating Spectrograms

In [2]:
def class_spectrogram(path):
    base = cv2.imread('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/channel__0.png')


    i = 1
    while i < 19:
        im = cv2.imread('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/channel__'+str(i)+'.png')
        base = cv2.vconcat([base,im])

        i += 1

    cv2.imwrite('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/class__0.png',base)


In [3]:
# raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/00002118_s007_t001.edf')

for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3/'+filename)
# print(filename)
    if raw is not None:
        for i in range(19):    
            spec,freq,t,im = plt.specgram(raw[i],Fs=100,NFFT=100,noverlap=50)
            plt.axis('off')
            figure = plt.gcf()
            figure.set_size_inches(12, 1)
            channel = 'channel__' + str(i)+'.png'
            plt.savefig(channel,bbox_inches = 'tight',pad_inches = 0)
            print(i,end=" ")

        class_spectrogram(3,filename[:-3])


Creating Signals


In [13]:
# raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_2/00001680_s005_t000.edf')

for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3/'+filename)
    if raw is not None:
        np.savetxt('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c3_csv/'+filename+'.csv', raw, delimiter=',')